In [2]:
import torch
import torch.nn as nn
from torch.nn import functional as F
# device = 'cuda' if torch.cuda.is_available() else 'cpu'
device = "mps" if torch.backends.mps.is_available() else "cpu"
print(device)
block_size = 8
batch_size = 4
max_iters = 1000
# eval_interval = 2500
learning_rate = 3e-4
eval_iters = 250

mps


In [3]:
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)


['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


In [4]:
string_to_int = { ch:i for i,ch in enumerate(chars) }
# print(string_to_int)
int_to_string = { i:ch for i,ch in enumerate(chars) }
# print(int_to_string)
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])
data = torch.tensor(encode(text), dtype=torch.long)
# print(data[:100])

In [6]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    print("ix:", ix)
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs:')
# print(x.shape)
print(x)
print('targets:')
print(y)



ix: tensor([ 23829,  55181, 150358, 111626])
inputs:
tensor([[ 1, 54, 73, 73, 58, 67, 73, 62],
        [73, 65, 78,  1, 68, 67,  1, 61],
        [73,  1, 78, 68, 74, 67, 60,  2],
        [58, 59, 74, 65, 65, 78,  1, 69]], device='mps:0')
targets:
tensor([[54, 73, 73, 58, 67, 73, 62, 68],
        [65, 78,  1, 68, 67,  1, 61, 58],
        [ 1, 78, 68, 74, 67, 60,  2,  3],
        [59, 74, 65, 65, 78,  1, 69, 65]], device='mps:0')


```python
ix = torch.randint(len(data) - block_size, (batch_size,))
```

This line generates random starting indices for sampling sequences (blocks) from the data. Let's break it down using the example `"Hello, how are you?"`.

## Understanding the Parameters

1. **`len(data) - block_size`**:
   - `len(data)` is the total number of elements (characters in this case) in the dataset.
     - In the string `"Hello, how are you?"`, the length (`len(data)`) is 19 (including spaces and punctuation).
   - `block_size` defines the number of consecutive elements you want to sample in one sequence (block). For example, let’s assume `block_size = 5`.
   - So, `len(data) - block_size` would be `19 - 5 = 14`. This is the maximum possible starting index for a sequence of 5 characters. If a sequence starts at index 14, it would go from character 14 to 18, which is the last valid block within the data.

2. **`(batch_size,)`**:
   - `batch_size` is the number of sequences (blocks) you want to sample in one batch. For instance, if `batch_size = 2`, you want to generate 2 random starting indices to get 2 blocks of data in this batch.

## Example Walkthrough

Assume:
- **`block_size = 5`**
- **`batch_size = 2`**
- **`data = "Hello, how are you?"`**

When you run:
```python
ix = torch.randint(len(data) - block_size, (batch_size,))
```
`torch.randint(14, (2,))` will generate 2 random integers between 0 and 13 (inclusive). These integers are used as starting indices for sampling blocks of size 5.  
Suppose `ix = [3, 10]`, the starting indices are 3 and 10.

## What Happens Next?

### For the first block starting at index 3:
The sequence will be:
```python
data[3:3+5] = "lo, h"
```
### For the second block starting at index 10:
The sequence will be:
```python
data[10:10+5] = "w are"
```

## Full Example

Let’s see what the code would do in this case:

- **`data = "Hello, how are you?"`**
- **Random indices** (`ix`): `[3, 10]`
- **`block_size = 5`**
- **`batch_size = 2`**

Now, for the input `x` and target `y`:

```python
x = torch.stack([data[i:i+block_size] for i in ix])
y = torch.stack([data[i+1:i+block_size+1] for i in ix])
```


- x (input sequences):
- x[0] = data[3:8] = "lo, h"
- x[1] = data[10:15] = "w are"
- y (target sequences, shifted by 1):
- y[0] = data[4:9] = "o, ho"
- y[1] = data[11:16] = " are "


In [7]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        """
        eval_iters determines how many batches will be used to estimate the loss for each dataset split 
        (e.g., 250 batches). This creates a more robust estimate than using a single batch.
        """
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [9]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)
        
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # print("Current context: ", index)
            # get the predictions
            logits, loss = self.forward(index)
            # print("Logits", logits)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
# print(generated_chars)



### Overview

The `BigramLanguageModel` class is a neural network model built using PyTorch's `nn.Module`. It is designed to work with a vocabulary of tokens (words or characters) and predicts the next token based on the current context using a bigram (two-token) approach.

### Components of the Class

1.  **Class Definition**:

    python

    Copy code

    `class BigramLanguageModel(nn.Module):`

    This line defines a new class called `BigramLanguageModel`, which inherits from `nn.Module`. This is a base class for all neural network modules in PyTorch.

2.  **`__init__` Method**:

    python

    Copy code

    `def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)`

    -   **Parameters**:
        -   `vocab_size`: The size of the vocabulary, which determines how many unique tokens the model can work with.
    -   **`super().__init__()`**: Calls the initializer of the parent class (`nn.Module`).
    -   **`self.token_embedding_table`**: This creates an embedding layer that maps each token to a dense vector of the same size as the vocabulary. This means that each token will be represented as a one-hot vector, but it can also be viewed as a dense embedding for the bigram context.
3.  **`forward` Method**:

    python

    Copy code

    `def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss`

    -   **Parameters**:
        -   `index`: A tensor of indices representing the input tokens.
        -   `targets`: Optional; a tensor of target indices for calculating loss.
    -   **Process**:
        -   The input indices are passed through the embedding layer to obtain `logits`, which represent the embeddings of the input tokens.
        -   If `targets` is not provided, `loss` is set to `None`.
        -   If `targets` is provided, the logits are reshaped to be suitable for loss calculation:
            -   `B`: Batch size
            -   `T`: Sequence length (number of tokens)
            -   `C`: Vocabulary size (number of classes)
        -   `logits` and `targets` are reshaped to 2D tensors so that they can be used for calculating cross-entropy loss.
        -   `F.cross_entropy(logits, targets)`: This computes the cross-entropy loss between the predicted logits and the actual targets.
    -   **Return Values**: The method returns the `logits` and the computed `loss`.
4.  **`generate` Method**:

    python

    Copy code

    `def generate(self, index, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self.forward(index)
            logits = logits[:, -1, :] # focus on the last time step
            probs = F.softmax(logits, dim=-1) # apply softmax to get probabilities
            index_next = torch.multinomial(probs, num_samples=1) # sample from the distribution
            index = torch.cat((index, index_next), dim=1) # append sampled index to the running sequence
        return index`

    -   **Parameters**:
        -   `index`: A tensor of shape `(B, T)` containing indices of the current context (where `B` is the batch size and `T` is the sequence length).
        -   `max_new_tokens`: The maximum number of new tokens to generate.
    -   **Process**:
        -   The function iteratively generates new tokens for `max_new_tokens` times:
            -   Calls the `forward` method to get the `logits`.
            -   `logits[:, -1, :]`: Focuses on the logits for the last time step, which represents the prediction for the next token.
            -   `F.softmax(logits, dim=-1)`: Applies softmax to convert logits into probabilities.
            -   `torch.multinomial(probs, num_samples=1)`: Samples a new token index from the probability distribution.
            -   `index = torch.cat((index, index_next), dim=1)`: Appends the sampled index to the running sequence.
    -   **Return Value**: Returns the updated `index` tensor, which now includes the newly generated tokens.

### Example Usage

python


`model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)`

1.  **Model Initialization**:

    -   `model = BigramLanguageModel(vocab_size)`: Initializes the model with a specified vocabulary size.
    -   `m = model.to(device)`: Moves the model to the specified device (CPU or GPU).
2.  **Context Initialization**:

    -   `context = torch.zeros((1, 1), dtype=torch.long, device=device)`: Initializes the context tensor to start generating text. It's a tensor with a single zero (indicating the first token in the vocabulary).
3.  **Text Generation**:

    -   `m.generate(context, max_new_tokens=500)`: Calls the `generate` method to produce up to 500 new tokens based on the initial context.
    -   `decode(...)`: This is presumably a function that converts the token indices back into a human-readable string format.
    -   `print(generated_chars)`: Outputs the generated characters.

### Summary

The `BigramLanguageModel` class is a simple yet effective language model that predicts the next token in a sequence based on the current context. It uses embeddings to represent tokens and has methods for both training (calculating loss) and inference (generating new sequences). The model can generate text based on the initial context and produces a sequence of tokens iteratively.

In [10]:
print(generated_chars)


[[&O3)FXYuD'e:57vJ?KZ!d3nmKToJ2O'B&ZpYv5g(57Q&QTjA-kP
JH5h[﻿ob6ijdHs6502ST5!*nm1'g!)o_Sk3OCHzxz
4[BBKc;mHii﻿w!3.'2Y&rdPw4
;0Bsdc;sjZO_L;Vv Q1""0UeA4luFme.ZDim),mh?_iGh;*T"AV3t-v7[bnJdeEfZW-jZW)pUjLHG;
LeVT!m"-t?-LWtDY&jc;.&;V_k(xol-d fZ!uzXtK'if(0vYrK-"&*_[*(VFKH" unm.4uhxbIYA?-qjZ:Vtx*Zc;oq-k"yhMUeOrf(a,sVPd,Uuv0vA8ID,
a9u5M,6wBL] QOILWMwNwmGR]ch;"Ws&-UZ h 32JnGR_SUc.27chx9Ue:ILW!uuD1meVlLnjf9.XgLbTtqme'gdoJ[)xD1c;VeE'&CCB9xE4pSL.hp;YLb4ju!uqFb:5N[w4pF 1?2d7gz[j,4Bkiy47uuy9!uD?7BwdsdW?_5yL:k:3g


In [13]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

    # sample a batch of data, returns random
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

ix: tensor([164371,  96779, 109279, 143527])
ix: tensor([151787,  69740, 183494, 132782])
ix: tensor([184372,  84692,  16777,  25383])
ix: tensor([ 14776, 100645, 165346,  97451])
ix: tensor([73948, 82309, 37653, 13934])
ix: tensor([169972, 172145, 116343, 162754])
ix: tensor([161348, 178976,  40311,  30845])
ix: tensor([ 61412, 107090, 135449, 127237])
ix: tensor([151845, 143726,  62560, 148471])
ix: tensor([176562,  68229,  60281,  45059])
ix: tensor([ 18593, 171148,  47846,  16987])
ix: tensor([ 68623,  40743, 139839,  68629])
ix: tensor([165034, 157011, 161176, 168152])
ix: tensor([146575, 169079, 173077, 102436])
ix: tensor([ 63262, 125717,  47171,  17011])
ix: tensor([139597,  60060, 129503,  29285])
ix: tensor([ 41391,  94279, 107936, 161440])
ix: tensor([173126,  37417,  40347,  52717])
ix: tensor([19118, 22568, 75016,  5547])
ix: tensor([ 14643, 130618,  25179,  63552])
ix: tensor([132068,  19692,  13686,  31575])
ix: tensor([153657, 146444, 105071,  38565])
ix: tensor([144267

***need to familiarize audience with optimizers (AdamW, Adam, SGD, MSE…) no need to jump into the formulas, just what the optimizer does for us and some of the differences/similarities between them***

1. **Mean Squared Error (MSE)**: MSE is a common loss function used in regression problems, where the goal is to predict a continuous output. It measures the average squared difference between the predicted and actual values, and is often used to train neural networks for regression tasks.
2. **Gradient Descent (GD):**  is an optimization algorithm used to minimize the loss function of a machine learning model. The loss function measures how well the model is able to predict the target variable based on the input features. The idea of GD is to iteratively adjust the model parameters in the direction of the steepest descent of the loss function
3. **Momentum**: Momentum is an extension of SGD that adds a "momentum" term to the parameter updates. This term helps smooth out the updates and allows the optimizer to continue moving in the right direction, even if the gradient changes direction or varies in magnitude. Momentum is particularly useful for training deep neural networks.
4. **RMSprop**: RMSprop is an optimization algorithm that uses a moving average of the squared gradient to adapt the learning rate of each parameter. This helps to avoid oscillations in the parameter updates and can improve convergence in some cases.
5. **Adam**: Adam is a popular optimization algorithm that combines the ideas of momentum and RMSprop. It uses a moving average of both the gradient and its squared value to adapt the learning rate of each parameter. Adam is often used as a default optimizer for deep learning models.
6. **AdamW**: AdamW is a modification of the Adam optimizer that adds weight decay to the parameter updates. This helps to regularize the model and can improve generalization performance. We will be using the AdamW optimizer as it best suits the properties of the model we will train in this video.

find more optimizers and details at torch.optim

In [14]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


-'rv2HrtRqfQ;V_W.KS﻿666GU39e(Hv7O8RXedxdcnP]c;eMe,Q﻿cYF Qg&qM&IDIWzuwb7CS4me.][D',
Kvi3sulgsP
-k&qNoPsCMPXJi,TFzohKypZbNVaSlB?K1oaE!Nq.xm KZ5meg(''5] feVT1mnsZbKa7c;_a8uMo6"Y,
97bwID2:n3F!qV[Jj,275y742utL',vf,rr*[QBv0V6"_.ua8VLxD'uUF)0,v7(QOt4.J2oW,st:7u4e[6EPF[*nTreH1g9!0C[JDiim*02Y"W4 BwcM-v0zFeH6NO1kCWF[_NEpGFXE&;8Cl'DW,lw_qGV_7cebu'ivjA?Z:h:jHS'v7c3:50qy?2F[EP*HE[yvmp(5braWf)GtlgI h4d[-d-JH68";Vchoh2 cYM7Bwnm1Fc;kxsRMGal-Otad,whchY9GCcMCj6tCVNM;s.sdH83LjO0x9QZO*Pg';T1?5(,4nmaqzGheyh3oJKeCDYP
